In [1]:
import pandas as pd
import os
import sys
import glob
sys.path.insert(0, '../')

### Import data for one stock

In [4]:
ph_df = pd.read_csv('../data/price-volume-data-for-all-us-stocks-etfs/Stocks/ph.us.txt')

In [8]:
ph_df[ph_df.Date == '2017-01-05']

,Date,Open,High,Low,Close,Volume,OpenInt
7944,2017-01-05,140.04,140.35,137.65,139.07,567140,0


### Run through stocks we pulled data from IEX for. Update the .csv file with this historical data

In [29]:
def add_historic_data_to_current_csv():
    current_data_file_path = '../data/historical_stock_data'
    augmented_data_file_path = '../data/updated_historical_stock_data'
    historic_data_file_path = '../data/price-volume-data-for-all-us-stocks-etfs/Stocks'

    for file in glob.glob(f'{current_data_file_path}/*.csv'):
        stock_name = file.rsplit("/")[-1].split('_')[0].lower() # note: store all stock names as uppercase
        # find the stock name in historical kaggle dataa
        for historical_file in glob.glob(f'{historic_data_file_path}/*.txt'):
            historical_stock_name = historical_file.split("/")[-1].split(".")[0]
            
            if stock_name == historical_stock_name:
                print(f"Found historical data for {stock_name}")
                # read in current data
                current_stock_data = pd.read_csv(f"{file}")
                historical_stock_data = pd.read_csv(f"{historical_file}")
                historical_stock_data.columns = map(str.lower, historical_stock_data.columns)
                print('')

                oldest_date_of_new_data = current_stock_data.date.min()

                # get the new data we don't have yet
                historical_stock_data[historical_stock_data.date < oldest_date_of_new_data]
                historical_stock_data.drop('openint', inplace=True, axis=1)
                print(historical_stock_data.date)
                print(current_stock_data.date)

                # concat historical with current data
                combined_stock_df = pd.concat([current_stock_data, historical_stock_data])
                print(combined_stock_df.groupby('date').count().sort_values(ascending=False, by='open'))
                most_recent_date = combined_stock_df.date.max()
                most_recent_year, most_recent_month, most_recent_day = most_recent_date.split('-')

                oldest_date = combined_stock_df.date.min()
                oldest_year, oldest_month, oldest_day = oldest_date.split('-')
                
                # save the file
                combined_stock_df.to_csv(f'{augmented_data_file_path}/{stock_name.lower()}_{int(oldest_year)}-{int(oldest_month)}-{int(oldest_day)}_to_{int(most_recent_year)}-{int(most_recent_month)}-{int(most_recent_day)}.csv',
                                         index=False)
                # delete the old file only if we haven't  updated it
                # os.remove(f"{file}")
                break

        # # if we don't have historical data , save the original data into the augmented folder
        # if os.path.isfile(f'{augmented_data_file_path}/{stock_name.lower()}_{int(oldest_year)}-{int(oldest_month)}-{int(oldest_day)}_to_{int(most_recent_year)}-{int(most_recent_month)}-{int(most_recent_day)}.csv'):
        #     pass
        # else:
        #     current_stock_data = pd.read_csv(f"{file}")
        #     current_stock_data.to_csv(f"{augmented_data_file_path}/)



In [30]:
add_historic_data_to_current_csv()

Found historical data for bac
0       1986-05-29
1       1986-05-30
2       1986-06-02
3       1986-06-03
4       1986-06-04
           ...    
7924    2017-11-06
7925    2017-11-07
7926    2017-11-08
7927    2017-11-09
7928    2017-11-10
Name: date, Length: 7929, dtype: object
0      2017-01-03
1      2017-01-04
2      2017-01-05
3      2017-01-06
4      2017-01-09
          ...    
823    2020-04-13
824    2020-04-14
825    2020-04-15
826    2020-04-16
827    2020-04-17
Name: date, Length: 828, dtype: object
            open  high  low  close  volume
date                                      
2017-11-06     2     2    2      2       2
2017-04-10     2     2    2      2       2
2017-04-26     2     2    2      2       2
2017-04-25     2     2    2      2       2
2017-04-24     2     2    2      2       2
...          ...   ...  ...    ...     ...
1997-08-20     1     1    1      1       1
1997-08-19     1     1    1      1       1
1997-08-18     1     1    1      1       1
1997-08-15 